# (WORKS) RAG2 : 
I'll try to use cloud sql (postgres) instead of local pgvector

In [4]:
from langchain_google_cloud_sql_pg import PostgresEngine
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresVectorStore
from unstructured.partition.md import partition_md
from unstructured.staging.base import dict_to_elements
from unstructured.chunking.title import chunk_by_title
import uuid

In [9]:
PROJECT_ID = "x-project-00"
REGION = "europe-west1"
INSTANCE = "vector-db"
DATABASE = "X3000_TurboFixer"
USER = "postgres"
PASSWORD = "admin"
TABLE_NAME = "vector_table2"

In [10]:
engine = await PostgresEngine.afrom_instance(
    project_id=PROJECT_ID, region=REGION, instance=INSTANCE, database=DATABASE, user=USER, password=PASSWORD
)

In [36]:
type(engine)

langchain_google_cloud_sql_pg.engine.PostgresEngine

In [11]:
await engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=768,  # Vector size for VertexAI model(textembedding-gecko@latest)
)

In [12]:
embeddings = VertexAIEmbeddings(model_name="text-embedding-004", project = PROJECT_ID)

In [13]:
store = await PostgresVectorStore.create(  # Use .create() to initialize an async vector store
    engine = engine,
    table_name = TABLE_NAME,
    embedding_service = embeddings,
)

In [22]:
partitioned_elements = partition_md(filename="..\\synthetic_data_generator\\doc_latest.md")
elements_dict = [el.to_dict() for el in partitioned_elements if el.category != "UncategorizedText"]

In [31]:
elements = dict_to_elements(elements_dict)

chunks = chunk_by_title(
    elements,
    combine_text_under_n_chars=800,
    max_characters=1500,
    # overlap=50
)

In [34]:
documents = []
ids = []
for element in chunks:
    metadatas = element.metadata.to_dict()
    del metadatas["languages"]
    metadatas["source"] = metadatas["filename"]
    documents.append(element.text)
    ids.append(str(uuid.uuid4()))


In [35]:
await store.aadd_texts(documents, metadatas=metadatas, ids=ids)


['3823d4d0-aac0-4b46-8695-2a0eeb5bff40',
 'fa397007-9ff5-4203-b5d7-ac6490f158a7',
 '41c5a950-7532-417b-a62d-61637b41c15c',
 'c4d9ebde-8cfe-4a2b-9b68-ad9fc3e64da6',
 'b1e06fff-5686-4a35-a9b9-56fd031f9d99',
 '161db09c-2df0-47ad-be91-900df50c64d8',
 '96819d35-5e32-4f87-929f-aededa695469',
 '649c146d-5c41-4f0b-bbf7-c24efc171a57',
 'ebbe122e-89b6-4205-9ac7-8f220f23930c',
 '36459c22-d6c8-4187-ab59-5e5b7077ea0c',
 '5ff69022-f3cb-4594-af87-001ed09c8ccc',
 'c015d5f1-326f-4c01-b31e-aa1c60d97b28',
 '0812134b-38ee-4b1e-bac9-5a5a5ab61c6e',
 'c72009a0-3791-43d2-b708-78fa42d249a7',
 '7c180505-3e01-4641-b854-b608925dbbb2',
 '80f6432a-c373-411d-8808-bca193c55cce']